In [2]:
""" 
Step 1:
"""

# 需要在 terminal 里运行起来，然后填补以下基本信息

mcp_url = "http://18.168.102.131:10254/sse" # master
mcp_name = "Agent To Agent MCP Server"
headers = {}

basic_mcp_info = {
    "mcp_name" : mcp_name,
    "sse_info": {
        "url": mcp_url,
        "headers": headers
    }
}

In [3]:
""" 
Step 2:
"""

from agents.mcp import MCPServerSseParams, MCPServerSse

def build_mcp_sse_server(mcp_info: dict):
    """ 
    example:
    {
        "mcp_name" : "",
        "sse_info": {
            "url": "https://your-server-url.com/mcp",
            "headers": {
                "Authorization": "Bearer your-token"
            }
        }
    }
    
    """
    
    mcp_sse_server = MCPServerSse(
        params=MCPServerSseParams(
            url=mcp_info['sse_info']['url'],
            timeout=200.0,  # 可选：HTTP 请求超时时间，默认 5 秒
            sse_read_timeout=300.0,  # 可选：SSE 连接超时时间，默认 5 分钟
            headers=mcp_info['sse_info']['headers'] 
        ),
        cache_tools_list=True,  # 可选：是否缓存工具列表
        name=mcp_info['mcp_name'],  # 可选：服务器名称
        client_session_timeout_seconds=10
    )
    
    return mcp_sse_server


mcp_server = build_mcp_sse_server(basic_mcp_info)
local_mcp_list=[mcp_server]

In [ ]:
""" 
Step 4:
"""

from agents import Runner 
import sys 
sys.path.append("..")
from src.stream_decoder import process_event_stream 

messages_list = [
]


In [ ]:

from contextlib import AsyncExitStack
import sys
sys.path.append('/root/liss_package/MXYZ-Agent-Core/src')
from src.stream_decoder import XYZAgentOutputDecoderType
from agents import Agent, OpenAIChatCompletionsModel
from openai import AsyncOpenAI 

async with AsyncExitStack() as stack:
    servers = []
    for mcp in local_mcp_list:
        server = await stack.enter_async_context(mcp)
        servers.append(server)
    db_agent = Agent(
        name = "Test Agent",
        instructions = "你是一个可以操作数据库的 AI-Agent，请充分利用你的 Tools 去帮助用户解决问题把。",
        mcp_servers=local_mcp_list,
        model = OpenAIChatCompletionsModel(
            model="gpt-4o-2024-11-20",
            openai_client=AsyncOpenAI(api_key="")   # TODO: 记得补充 api key
            )
    )

    this_time_input = "你能帮我问问Agent  1037  现在在做什么吗？这是他的 a2a url：  http://127.0.0.1:5000/1037"



    user_input = {
        "role": "user",
        "content": this_time_input 
    }

    messages_list.append(user_input)

    response = Runner.run_streamed(db_agent, messages_list)

    mind_log = []
    async for event in process_event_stream(response):
        if event.stream_show:
            print(event.data.content, end="")
            # mind_log.append(event) 
        else:
            mind_log.append(event) 
    this_time_response_content = ""
    print("")
    for event in mind_log:
        print(event)
        if event.type == XYZAgentOutputDecoderType.MESSAGE_CONTENT:
            this_time_response_content = event.data.content
    messages_list.append({
        "role": "assistant",
        "content": this_time_response_content
    })

我无法通过给定的 URL 成功获取 Agent 1037 的信息。可能是以下原因之一：

1. URL 输入存在问题。
2. Agent Server 服务可能出现问题。

请确认 URL 的正确性，或确认 Agent Server 是否正常运行。
type=<XYZAgentOutputDecoderType.AGENT_CALL: 'agent_call'> data=AgentCallOutput(content=None, mcp_server_name=None, mcp_tool_name=None, mcp_tool_arguments=None, mcp_tool_response=None, agent_name='Test Agent', agent_arguments={'instructions': '你是一个可以操作数据库的 AI-Agent，请充分利用你的 Tools 去帮助用户解决问题把。', 'model': <agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7fc6551a1420>, 'available_tools': []}, interaction_chain=[], power_name=None, power_arguments=None, power_response=None, tool_name=None, tool_arguments=None, tool_response=None, usage_report=None, signal_type=None, signal_data=None) stream_show=False new_agent=Agent(name='Test Agent', instructions='你是一个可以操作数据库的 AI-Agent，请充分利用你的 Tools 去帮助用户解决问题把。', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7fc6551a1420>, model_set